##### 0) 라이브러리 불러오기

In [1]:
# (1) 기본 라이브러리
import numpy as np

# (2) 머신러닝 라이브러리
import torch
import torch.optim as optim

In [2]:
torch.manual_seed(0)

##### 1) 데이터 불러오기

In [3]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.") # !!

##### 2) Unique 단어장 생성

In [4]:
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [5]:
char_set

['u',
 'w',
 ',',
 'i',
 'f',
 'c',
 'b',
 's',
 "'",
 'm',
 'y',
 '.',
 'a',
 'd',
 'k',
 'h',
 'r',
 ' ',
 'o',
 't',
 'e',
 'n',
 'g',
 'l',
 'p']

In [6]:
char_dic

{'u': 0,
 'w': 1,
 ',': 2,
 'i': 3,
 'f': 4,
 'c': 5,
 'b': 6,
 's': 7,
 "'": 8,
 'm': 9,
 'y': 10,
 '.': 11,
 'a': 12,
 'd': 13,
 'k': 14,
 'h': 15,
 'r': 16,
 ' ': 17,
 'o': 18,
 't': 19,
 'e': 20,
 'n': 21,
 'g': 22,
 'l': 23,
 'p': 24}

##### 3) 하이퍼파라미터 정의

In [7]:
input_size = len(char_dic) # dic_size = input_size
hidden_size = len(char_dic)
sequence_length = 10 # !!
learning_rate = 0.1

In [8]:
input_size

25

In [9]:
hidden_size

25

##### 4) 데이터 전처리

(1) String -> Int

(2) Data Split

In [10]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length): # len(sentence) - sequence_length까지 순회하는게 아니라, len(sentence) - sequence_length - 1까지만 순회함 !!!
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1:i + sequence_length + 1] # !!!
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str]) # x str to index # !
    y_data.append([char_dic[c] for c in y_str]) # y str to index # !

x_one_hot = [np.eye(input_size)[x] for x in x_data] # !!!

# print(f"np.array(x_data).shape : {np.array(x_data).shape}") # (170, 10)
# print(f"np.array(y_data).shape : {np.array(y_data).shape}") # (170, 10)
# print(f"np.array(x_one_hot).shape : {np.array(x_one_hot).shape}") # (170, 10, 25)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [30]:
np.array(x_data).shape

(170, 10)

In [29]:
np.array(y_data).shape

(170, 10)

In [28]:
np.array(x_one_hot).shape

(170, 10, 25)

(3) Numpy -> Tensor

In [20]:
x_train = torch.FloatTensor(x_one_hot)
y_train = torch.LongTensor(y_data)

C:\Users\Bang\AppData\Local\Temp\ipykernel_13516\1151046340.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  x_train = torch.FloatTensor(x_one_hot)


In [22]:
x_train.shape

torch.Size([170, 10, 25])

In [23]:
y_train.shape

torch.Size([170, 10])

In [24]:
# declare RNN + FC
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__() # !!
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True) # !!!
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True) # hidden 부분이 아닌, output을 보내는 부분에 linear 추가한 것 !!

    def forward(self, x):
        x, _status = self.rnn(x) # !!
        x = self.fc(x) # !!
        return x

net = Net(input_size, hidden_size, 2) # layer !!

##### 6) Optimizer + Loss Function 정의

In [25]:
criterion = torch.nn.CrossEntropyLoss() # !
optimizer = optim.Adam(net.parameters(), learning_rate) # !

##### 7) 모델 학습

In [29]:
# (1) Epoch 순회
for i in range(100):
    # 1] 모델 예측
    y_pred = net(x_train) # outputs = y_pred
    """
    print(f"y_pred.size() : {y_pred.size()}") # torch.Size([170, 10, 25])
    print(f"input_size : {input_size}") # 25
    print(f"y_pred.view(-1, input_size).size() : {y_pred.view(-1, input_size).size()}") # torch.Size([1700, 25])
    print(f"y_pred.view(-1, input_size)[0] : {y_pred.view(-1, input_size)[0]}") # tensor([-0.1648,  0.1511,  0.0908,  0.3193, -0.0533, -0.1044, -0.1668,  0.3143, 0.0613, -0.0017, -0.2346,  0.0379,  0.3102,  0.1924, -0.0051, -0.0549, -0.1215, -0.0656,  0.1834,  0.0720, -0.2839, -0.2851,  0.0314,  0.0024, 0.0470], grad_fn=<SelectBackward0>)
    print(f"y_train.view(-1).size : {y_train.view(-1).size()}") # torch.Size([1700])
    """
    # 2] Cost 계산
    loss = criterion(y_pred.view(-1, input_size), y_train.view(-1)) # torch.Size([1700, 25]), torch.Size([1700])
    # 3] 모델 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 4] 예측값 Int
    batch_pred_data_int = y_pred.argmax(dim=2)
    """
    print(f"y_pred.data.numpy().shape : {y_pred.data.numpy().shape}") # (170, 10, 25)
    print(f"batch_pred_data_int : {batch_pred_data_int}")
    tensor([[ 3,  3, 13,  ...,  7, 13,  7],
        [12, 13, 13,  ..., 13,  7, 13],
        [ 3, 13,  7,  ...,  7, 13,  7],
        ...,
        [12, 13, 13,  ..., 13, 18,  7],
        [ 3, 13,  3,  ..., 18,  7,  7],
        [12,  3, 13,  ...,  7,  7,  7]])
    print(f"batch_pred_data_int.shape : {batch_pred_data_int.shape}") # torch.Size([170, 10])
    print(f"np.squeeze(batch_pred_data_int).shape : {np.squeeze(batch_pred_data_int).shape}") # torch.Size([170, 10])
    """
    # 5] 예측값 Str
    batch_pred_data_str = ""
    for j, pred_data_int in enumerate(batch_pred_data_int):
        # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        # 첫 예측 : 예측 결과 전부 가져오기 !!!
        if j == 0:
            batch_pred_data_str += ''.join([char_set[t] for t in pred_data_int])
        # 이후 예측 : 이전 예측 결과와 중첩되는 부분을 제외하고, 예측 결과의 마지막 부분만 가져오기 !!!
        else:
            batch_pred_data_str += char_set[pred_data_int[-1]]

    # print("epoch : ", i, "loss : ", loss.item(), "batch_pred_data_int : ", batch_pred_data_int, "true_data : ", y_data)
    print("batch_pred_data_str : ", batch_pred_data_str)

batch_pred_data_str :  g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
batch_pred_data_str :  t you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
batch_pred_data_str :  m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
batch_pred_data_str :  m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
batch_pred_data_str :  f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensi